<a href="https://colab.research.google.com/github/praisemwanza53/Open-Source-LLMs/blob/main/Climate_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Climate Data Analysis and Forecasting**
Weather Prediction Models: Utilize historical weather data and real-time inputs to predict short-term and long-term weather patterns. This could include temperature, rainfall, humidity, or extreme events like hurricanes.
Climate Trend Analysis: Analyze climate change data trends, such as global temperature increases, sea-level rise, and CO2 emissions over time.

*A. Input Variables*
These variables include both historical and real-time data that the AI model will use to predict future weather conditions.

Temperature (°C): Historical and real-time temperature data.
Humidity (%): Percentage of moisture in the air (historical and real-time).
Wind Speed (km/h or m/s): Historical and real-time wind data.
Precipitation (mm): Rainfall or snow levels.
Pressure (hPa): Atmospheric pressure data, which can affect weather predictions.
Cloud Cover (%): Percentage of cloud cover in the sky.
Solar Radiation (W/m²): Solar energy received at the surface (optional, depending on the data source).
Historical Weather Data: A dataset of past weather observations for a region, which will be fed into the model along with real-time data.

*B. Output Variables*
These are the predictions made by the AI model:

Predicted Temperature (°C): Temperature forecast for future dates.
Predicted Humidity (%): Humidity forecast for future dates.
Predicted Wind Speed (km/h or m/s): Wind speed forecast for future dates.
Predicted Precipitation (mm): Expected rainfall or snowfall.
Extreme Weather Alerts: Predictions for hurricanes, storms, or other severe weather patterns.

Data from Major Cities by Province:
Lusaka (Lusaka Province)
Ndola (Copperbelt Province)
Livingstone (Southern Province)
Chipata (Eastern Province)
Solwezi (North-Western Province)
Mansa (Luapula Province)
Kasama (Northern Province)
Mongu (Western Province)
Kabwe (Central Province)
Chinsali (Muchinga Province)

**2. Energy and Resource Optimization**
Renewable Energy Forecasting: AI models to predict the availability of solar or wind energy based on weather conditions. For example, forecasting solar panel efficiency based on cloud coverage predictions.
Energy Demand Prediction: Models to predict energy demand patterns in different regions depending on seasonal climate changes, optimizing energy usage based on predictions.

**3. Climate Change Impact Modeling**
Ecosystem Impact Models: Models that predict how changing temperatures, weather patterns, and CO2 levels affect ecosystems, plant growth, and species migration.
Agricultural Yield Predictions: AI models to predict how climate changes (like temperature and rainfall variation) will impact crop yields in different regions.

** 4. Climate Education and Awareness**
Chatbots and Virtual Assistants: Building conversational agents that educate users on climate change, suggest lifestyle changes, and provide real-time data on climate conditions and tips for sustainability.
AI for Environmental Science Research: Automating climate-related research tasks like extracting data from scientific papers or predicting the outcomes of complex climate models.

**1. Climate Data Analysis and Forecasting**

Cell 1: Install Required Libraries

In [ ]:
# Install required libraries
!pip install meteostat
!pip install scikit-learn


Cell 2: Import Libraries



In [ ]:
# Import necessary libraries
import pandas as pd
import requests
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


Cell 3: Define API Key and Major Cities in Zambia

In [ ]:
# Define your OpenWeather API key
API_KEY = '5f1c8cc40140a0216562c119e047d5c5'  # Replace with your OpenWeather API key

# Define the list of major cities in Zambia with their respective coordinates
cities = {
    "Lusaka": {"lat": -15.3875, "lon": 28.3228},
    "Ndola": {"lat": -12.9587, "lon": 28.6366},
    "Livingstone": {"lat": -17.8587, "lon": 25.8562},
    "Chipata": {"lat": -13.6333, "lon": 32.6500},
    "Solwezi": {"lat": -12.1833, "lon": 26.4000},
    "Mansa": {"lat": -11.1997, "lon": 28.8940},
    "Kasama": {"lat": -10.2129, "lon": 31.1790},
    "Mongu": {"lat": -15.2484, "lon": 23.1274},
    "Kabwe": {"lat": -14.4464, "lon": 28.4465},
    "Chinsali": {"lat": -10.5500, "lon": 32.0667}
}


Cell 4: Fetch Historical Weather Data from OpenWeather API

In [ ]:
# Function to fetch historical weather data for the past 10 years
def fetch_historical_weather(city, lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine"
    historical_data = []

     # Fetch data for each day in the past 3 years
    for days_ago in range(1, 1096):  # 1095 days = 3 years
        dt = int((datetime.now() - timedelta(days=days_ago)).timestamp())
        response = requests.get(url, params={
            'lat': lat,
            'lon': lon,
            'dt': dt,
            'appid': API_KEY,
            'units': 'metric'  # Use 'imperial' for Fahrenheit
        })
        data = response.json()

        if 'current' in data:
            historical_data.append({
                'city': city,
                'tavg': data['current']['temp'],
                'tmin': data['current']['temp_min'],
                'tmax': data['current']['temp_max'],
                'prcp': data.get('current', {}).get('precipitation', 0),  # Adjust for API structure
                'wspd': data['current']['wind_speed']
            })

    return historical_data

# Collect historical data for all cities
all_historical_data = []
for city, coords in cities.items():
    data = fetch_historical_weather(city, coords['lat'], coords['lon'])
    all_historical_data.extend(data)

# Create DataFrame from historical data
all_data = pd.DataFrame(all_historical_data)

# Display a sample of the combined historical data
print(all_data.head())


Empty DataFrame
Columns: []
Index: []


Cell 5: Data Preprocessing

In [ ]:
# Clean the combined historical data
all_data = all_data.dropna()  # Drop rows with missing values

# Select relevant features
X = all_data[['tmin', 'tmax', 'prcp', 'wspd']]  # Features: min/max temperature, precipitation, wind speed
y = all_data['tavg']  # Target: average temperature

# Display a preview of the preprocessed data
print(X.head(), y.head())


Cell 6: Train-Test Split

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display sizes of the training and test sets
print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")


Cell 7: Train AI Model (Random Forest Regressor)

In [ ]:
# Initialize the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Confirm model training
print("Model training completed!")


Cell 8: Model Evaluation

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance using Mean Squared Error (MSE) and Root Mean Squared Error (RMSE)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Display evaluation results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Display actual vs predicted results
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results.head())


Cell 9: Fetch Real-Time Data for Zambia

In [ ]:
# Function to fetch current weather data for each city
def fetch_current_weather(city, lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/weather"
    response = requests.get(url, params={
        'lat': lat,
        'lon': lon,
        'appid': API_KEY,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    })
    return response.json()

# Fetch real-time weather data for each city
real_time_data = []
for city, coords in cities.items():
    data = fetch_current_weather(city, coords['lat'], coords['lon'])
    real_time_data.append({
        'city': city,
        'temp': data['main']['temp'],
        'wspd': data['wind']['speed'],
        'prcp': data.get('rain', {}).get('1h', 0)  # Precipitation in the last hour, if available
    })

# Create DataFrame from real-time data
real_time_df = pd.DataFrame(real_time_data)

# Display real-time weather data
print(real_time_df)


Cell 10: Make Prediction Based on Real-Time Data

In [ ]:
# Prepare real-time data for prediction
real_time_input = pd.DataFrame({
    'tmin': real_time_df['temp'] - 2,  # Assuming min temp is 2°C lower than the observed temperature
    'tmax': real_time_df['temp'] + 2,  # Assuming max temp is 2°C higher than the observed temperature
    'prcp': real_time_df['prcp'],  # Real-time precipitation
    'wspd': real_time_df['wspd']  # Real-time wind speed
})

# Make prediction using the trained model
real_time_prediction = model.predict(real_time_input)

# Display the predicted temperature based on real-time data
for i, city in enumerate(real_time_df['city']):
    print(f"Predicted average temperature for {city}: {real_time_prediction[i]:.2f}°C")
